# Executing commands and Playbooks at Nodes and using Role System

This notebook  brief describes how to execute commands in CLAP nodes. There are three ways to execute commands:
* Directly executing shell commands
* Executing an Ansible playbook
* Executing role's actions

In [9]:
import sys
sys.path.append('../..')
!pip list

Package               Version
--------------------- -----------
ansible               4.0.0
ansible-core          2.11.1
ansible-runner        1.4.7
argcomplete           1.12.3
attrs                 21.2.0
backcall              0.2.0
bcrypt                3.2.0
boto                  2.49.0
boto3                 1.17.89
botocore              1.20.89
cffi                  1.14.5
click                 8.0.1
coloredlogs           15.0
contextlib2           0.6.0.post1
cryptography          3.4.7
cycler                0.10.0
dacite                1.6.0
decorator             5.0.9
docutils              0.17.1
fire                  0.4.0
humanfriendly         9.1
iniconfig             1.1.1
ipaddress             1.0.23
ipykernel             5.5.5
ipython               7.24.1
ipython-genutils      0.2.0
jedi                  0.18.0
Jinja2                3.0.1
jmespath              0.10.0
jupyter-client        6.1.12
jupyter-core          4.7.1
kiwisolver            1.3.1
lockfile             

Let's perform traditional imports

In [11]:
import yaml
from dataclasses import asdict
from app.cli.modules.node import get_config_db, get_node_manager
from app.cli.modules.role import get_role_manager
from clap.utils import float_time_to_string, path_extend
from clap.executor import SSHCommandExecutor, AnsiblePlaybookExecutor

Let's create manager objects

In [13]:
configuration_db = get_config_db()
node_manager = get_node_manager()
role_manager = get_role_manager()
# Private's path (usually ~/.clap/private/) will be used for other methods
private_path = node_manager.private_path

And get nodes in CLAP's system. Here we have 2 nodes, previously created with CLAP

In [14]:
nodes = node_manager.stoget_all_nodes()
for node in nodes:
    print(f"Node ID: {node.node_id} ({node.nickname}); status: {node.status}; "
          f"IP: {node.ip}; type: {node.configuration.instance.instance_config_id}")

Node ID: 3363c82c3cbb42ab89ce81204de15334 (PatriciaNoble); status: reachable; IP: 100.27.0.240; type: type-small
Node ID: e926ed7a84e84d6a8009063321646f8b (TimTaylor); status: reachable; IP: 52.90.15.82; type: type-small
Node ID: e1b978794d5b46499e57a6db33e7c068 (InaHagen); status: reachable; IP: 100.25.119.221; type: type-small
Node ID: 4bdf519cdffa4599a4f1f086967a19bf (JanetGibson); status: reachable; IP: 18.204.15.129; type: type-small
Node ID: a056ed6e721541f3874b1ff1fa87f29a (DorothyGrayson); status: reachable; IP: 54.86.73.220; type: type-small
Node ID: 3b8159b1da7547b88b900f3c6fbb3658 (KatharineBlock); status: reachable; IP: 54.160.67.247; type: type-small
Node ID: d1a8cf404d824e25bc139e338f2925cd (SandraSlaughter); status: reachable; IP: 54.160.70.178; type: type-small
Node ID: 00ffaa3eb91e4622b1dcff6e05244527 (DavidWilburn); status: reachable; IP: 100.26.232.51; type: type-small
Node ID: 7448cb59df3f4831b61f633c8ace4a20 (LeonardGillies); status: reachable; IP: 54.236.16.251; t

Let's update node information using IS alive method

In [15]:
node_ids = [node.node_id for node in nodes]
for node_id, status in node_manager.is_alive(node_ids).items():
    print(f"Node {node_id} is {'alive' if status else 'not alive'}")

Error checking nodes 3363c82c3cbb42ab89ce81204de15334,e926ed7a84e84d6a8009063321646f8b,e1b978794d5b46499e57a6db33e7c068,4bdf519cdffa4599a4f1f086967a19bf,a056ed6e721541f3874b1ff1fa87f29a,3b8159b1da7547b88b900f3c6fbb3658,d1a8cf404d824e25bc139e338f2925cd,00ffaa3eb91e4622b1dcff6e05244527,7448cb59df3f4831b61f633c8ace4a20,885bad467fd04dfca6aa4a47fe52c25f,07b06ea7d7b5403ebc719af9899bad0e,2ab09bd867814886b7d931918d142229,47bf901d41624d7d9fa08233a97d1972,561b7392f5114bdab6ad3e9a82d32566,530355a73e8442169b91a111bf2d5312,2dac7edf679049b0a1ffbed9b3aa0d63,6b176c1a263b41d4ab67fc6e4026d388,235d027c961f417baf982ae96a0353e9,0fa652b653854ee5811bd2a863d12861,3cd37d7f0a4141f893edbbb5bdb6f651,ad4dc9ed3bb14a41bf491db39ed6dfdc,8aae0349f6ba4d7aa96892ad0b869758,821f049cb66c49c0a8c97f37f93dd151,b2711c8444c6469391550cc17be18c1f,581bb72e9a2346dc8e17bd738033d453,b2075132869946c28e2aca9382c8ea4e,aff3c44b3780426abecc8056eac646bb,a5ab86a874484f0394671bac46aaf289,fa021323708d4ecc8bf94a76ddf66cf3,a0c140b7b09646e4933bb8

Exception: Update task returned no events to process

## Executing Shell commands directly, using SSHCommandExecutor class

The SSHCommandExecutor class allows you to execute shel commands in nodes. The command is a string of shell commands. This class must be initializated with the command string, the list of **NodeDescriptors** and the path to private directory.

After using the run() method from SSHCommandExecutor, it wil be returned a dataclass called `CommandResult` with the following information:
* ok: if the command was executed in nodes (SSH performed and command executed)
* ret_code: if the command was executed (ok==True), this will contain the return code, otherwise None
* stdout_lines: if the command was executed (ok==True), this will contain a list of strings of the stdout output, otherwise None
* stderr_lines: if the command was executed (ok==True), this will contain a list of strings of the stderr output, otherwise None
* error: if the command was not executed (ok==False), this will contain the string with the exception, otherwise None

In [6]:
command_to_execute = """
git clone https://github.com/lmcad-unicamp/CLAP.git CLAP-ssh
echo Clonned CLAP into CLAP-ssh
"""
executor = SSHCommandExecutor(command_to_execute, nodes, private_path)
result = executor.run()

for node_id, res in result.items():
    print(f"Node id {node_id}, executed the command: {res.ok}, ret code: {res.ret_code}")
    # resut is a dataclass, we can convert to a dictionary
    res_dict = asdict(res)
    print('-----')
    # Dump dictionary in YAML format
    print(yaml.dump(res_dict, indent=4, sort_keys=True))

Node id 07f4a369663f48d48254f2ad4c5abbfe, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- 'Cloning into ''CLAP-ssh''...

    '
stdout_lines:
- 'Clonned CLAP into CLAP-ssh

    '

Node id 8133c6f7f9ca48258d1e0f01e11326f6, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- 'Cloning into ''CLAP-ssh''...

    '
stdout_lines:
- 'Clonned CLAP into CLAP-ssh

    '



# Executing a Ansible playbok in nodes, using AnsiblePlaybookExecutor class

AnsiblePlaybookExecutor class allows CLAP to execute an Ansible playbook directly in CLAP's nodes. This class must be initializated with the path of the playbook, the path to private directory and the inventory.

After using the run() method from AnsiblePlaybookExecutor, it will be returned a dataclass called `PlaybookResult` with the following information:
* ok: if the command was executed in nodes (playbook was executed)
* ret_code: Ansible return code
* hosts: A dictionary where the keys are the node ids and the value is a boolean containing True if playbook was executed without errors in node and false otherwise.
* events: A dictionary where the keys are the node ids and the value is a list of dicionaries with all ansible events (see ansible-runner the know the events)
* vars: A dictionary where the keys are the node ids and the value is a dictionary of the facts set to this node inside the playbook. So, every set_fact in Ansible Playbook is visible to CLAP through this vvariable. 

The inventory can be generated using AnsiblePlaybookExecutor.create_inventory static method, passing the nodes and the private path as arguments.

We will execute the playbook listed bellow at nodes: `07f4a369663f48d48254f2ad4c5abbfe` and `8133c6f7f9ca48258d1e0f01e11326f6`. The playbook can be used to install CLAP at nodes.

The playbook: 
* Update apt cache and install packages
* Clone a git repository
* Set CLAP's install.sh file to be executable
* Run install.sh script
* Set a fact called clap_dir with the directory where CLAP was installed at remote hosts. It will be visible to PlaybookResult.vars

Besides that, variables can be passed to playbook through 'extra' variable when initiaizing AnsiblePlaybookExecutor. This parameter receives a dictionary with key and values being strings.

In [7]:
!cat ~/CLAP/getfacts.yml

---
- hosts: all
  gather_facts: True    # Query a set of variables in remote hosts
  gather_subset: min
  tasks:
  - name: Get iteration time from a fact
    get_fact:
      interation_time: "{{ ansible_local.times.iteration_time }}"

In [17]:
playbook_file = path_extend('~/CLAP/getfacts.yml')
inventory = AnsiblePlaybookExecutor.create_inventory(nodes, private_path)
executor = AnsiblePlaybookExecutor(playbook_file, private_path, inventory=inventory)
result = executor.run()

 "unreachable": true}

TASK [Get iteration time from a fact] ******************************************
ok: [d4289a6df8f4462c9de952b2c95dc817]
fatal: [0a6d71d1846f4085a3e7b433854d8385]: FAILED! => {"msg": "The task includes an option with an undefined variable. The error was: 'dict object' has no attribute 'times'\n\nThe error appears to be in '/home/roberto/CLAP/getfacts.yml': line 6, column 5, but may\nbe elsewhere in the file depending on the exact syntax problem.\n\nThe offending line appears to be:\n\n  tasks:\n  - name: Get iteration time from a fact\n    ^ here\n"}
fatal: [2cf810aaf68f4b1fa9d8ac2cb48dd002]: FAILED! => {"msg": "The task includes an option with an undefined variable. The error was: 'dict object' has no attribute 'times'\n\nThe error appears to be in '/home/roberto/CLAP/getfacts.yml': line 6, column 5, but may\nbe elsewhere in the file depending on the exact syntax problem.\n\nThe offending line appears to be:\n\n  tasks:\n  - name: Get iteration time from a fact\n

Let's check the playbook results

In [18]:
print(f"Did the playbook executed? {result.ok}")
print(f"Ansible playbook return code: {result.ret_code}")
print(f"Let's check how nodes executed: ")
for node_id, status in result.hosts.items():
    print(f"    Node {node_id}: {status}")
print(f"Let's check variables set using set_fact module: ")
for node_id, facts in result.vars.items():
    print(f"    Node {node_id}: {facts}")

Did the playbook executed? False
Ansible playbook return code: 4
Let's check how nodes executed: 
    Node cf1be71c4cdc4197bbce6c61bf261fc2: False
    Node 824fd6c984a94dea89d0132391f01cff: False
    Node 2dac7edf679049b0a1ffbed9b3aa0d63: False
    Node e926ed7a84e84d6a8009063321646f8b: False
    Node fa021323708d4ecc8bf94a76ddf66cf3: False
    Node ca2504f7954f46249bdef76173805565: False
    Node a056ed6e721541f3874b1ff1fa87f29a: False
    Node bb91451d2abf48c5829f2405202580d9: False
    Node cf7b1a6f0292431ba034c8c10552ed8d: False
    Node 4ad77561809e4e88aa90acc35cb19fcb: False
    Node ad4dc9ed3bb14a41bf491db39ed6dfdc: False
    Node 561b7392f5114bdab6ad3e9a82d32566: False
    Node 181b3274b6754bd0b16e34a71dc33d3f: False
    Node 48a545c110974591a2599404c7278605: False
    Node 3b8159b1da7547b88b900f3c6fbb3658: False
    Node aff3c44b3780426abecc8056eac646bb: False
    Node e882ffe98c2d498da09d937a2f3b4a4a: False
    Node 2d76e4b5256747a3ba67351faa64ddb2: False
    Node 3cd37d7f0a4

# Roles

Roles are easy ways to organize and execute Playbook at nodes that play a role. We will add nodes to a role called `commands-common`. Nodesthat play this role can perform common operations like copy and fetch files, install and update packages, reboot, among others.

We will copy files from local to remote hosts, using commands-common role. The workflow will be:
* Add nodes to commands-common role
* Execute action calle copy on nodes of role commands-common

Let's see the commands common role at actions.d. The copy action requires the source file and the destiny where the files will be placed at remote hosts.

In [10]:
!cat ~/.clap/roles/actions.d/commands-common.yaml

---
actions:
  install-packages:
    playbook: roles/commands-common_install.yml
    description: Install packages in nodes
    vars:
    - name: packages
      description: Packages to install (comma separated)

  copy:
    playbook: roles/commands-common_copy.yml
    description: Copy files from localhost to remote hosts
    vars:
    - name: src
      description: Source files/directory to be copied
    - name: dest
      description: Destination directory where files will be placed

  fetch:
    playbook: roles/commands-common_fetch.yml
    description: Fetch files from remote hosts to localhosts
    vars:
    - name: src
      description: Source files/directory to be fetched
    - name: dest
      description: Destination directory where files will be placed

  reboot:
    playbook: roles/commands-common_reboot.yml
    description: Reboot a machine

  run-command:
    playbook: roles/commands-common_run-command.yml
    description: Run a shell command in remote hosts
    vars:
  

All roles can be accessed though `roles` dictionary from `role_manager` class, where the keys are the role names and values are the `Role` dataclass. We will print the role commands-common as a dict.

In [11]:
for role_name, role_info in role_manager.roles.items():
    # Convert role_info to a dict
    role_dict = asdict(role_info)
    print('------')
    print(f"Role: {role_name}")
    # Print role
    print(f"{yaml.dump(role_dict, indent=4)}")

------
Role: commands-common
actions:
    copy:
        description: Copy files from localhost to remote hosts
        playbook: roles/commands-common_copy.yml
        vars:
        -   description: Source files/directory to be copied
            name: src
            optional: false
        -   description: Destination directory where files will be placed
            name: dest
            optional: false
    fetch:
        description: Fetch files from remote hosts to localhosts
        playbook: roles/commands-common_fetch.yml
        vars:
        -   description: Source files/directory to be fetched
            name: src
            optional: false
        -   description: Destination directory where files will be placed
            name: dest
            optional: false
    install-packages:
        description: Install packages in nodes
        playbook: roles/commands-common_install.yml
        vars:
        -   description: Packages to install (comma separated)
            nam

Let's add nodes `07f4a369663f48d48254f2ad4c5abbfe` and `8133c6f7f9ca48258d1e0f01e11326f6` to commands-comon role.

**Note**: If there is an action named `setup` defined in role, when using `add_role` from `RoleManager` class, this action will be automatically executed. The nodes will not be added to nodes if this action fails. If this action does not exists, the nodes will be added to role only.

In [12]:
added_nodes = role_manager.add_role('commands-common', node_ids)
print(f"Role commands-common was added to {len(added_nodes)} nodes: {node_ids}")

Role commands-common was added to 2 nodes: ['07f4a369663f48d48254f2ad4c5abbfe', '8133c6f7f9ca48258d1e0f01e11326f6']


Let's perform action update-packages at nodes and copy the file at `~/playbook.yml` to remote hosts, at `~`. 
The copy action requires `src` and `dest` vars to be informed. It will be informed through `extra_args` variable.

**Note**: An error will be raised if the variables are not informed 

In [13]:
copy_vars = {
    'src': path_extend('~/playbook.yml'),
    'dest': '~'
}
playbook_result = role_manager.perform_action('commands-common', 'update-packages', node_ids)
playbook_result = role_manager.perform_action('commands-common', 'copy', node_ids, extra_args=copy_vars)

[WARNING]: Invalid characters were found in group names but not replaced, use
-vvvv to see details

PLAY [all] *********************************************************************

TASK [Perform package list update] *********************************************
changed: [07f4a369663f48d48254f2ad4c5abbfe]
changed: [8133c6f7f9ca48258d1e0f01e11326f6]
07f4a369663f48d48254f2ad4c5abbfe : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
8133c6f7f9ca48258d1e0f01e11326f6 : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

PLAY RECAP *********************************************************************
07f4a369663f48d48254f2ad4c5abbfe : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
8133c6f7f9ca48258d1e0f01e11326f6 : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

[WARNING]: Invalid characters were found in group names but not replac

We can also check all nodes that belong to a particular role using `get_all_role_nodes` method from `RoleManager` class.

If your role define hosts, you can use `get_all_role_nodes_hosts`.

In [14]:
nodes_belonging_to_role = role_manager.get_all_role_nodes('commands-common')
print(nodes_belonging_to_role)

['07f4a369663f48d48254f2ad4c5abbfe', '8133c6f7f9ca48258d1e0f01e11326f6']
